In [ ]:
%matplotlib widget
from bmcs_shell.api import WBCell5Param, WBCell5ParamV2, WBCell5ParamV3, WBCell4Param
import numpy as np
from numpy import sqrt, sin, cos, tan, arctan

# Demo showing kinematics of different waterbomb cells

## Waterbomb cells with 5 parameters (asymmetric cell)

### Cell V1

In [ ]:
wb = WBCell5Param()
wb.interact()

### Cell V2

In [ ]:
wb = WBCell5ParamV2()
wb.interact()

### Cell V3

In [ ]:
wb = WBCell5ParamV3(debug=True)
wb.interact()

In [ ]:
diff = wb.X_Ia[5]-wb.X_Ia[0]
print(diff)
np.sqrt(np.einsum('a, a', diff, diff))

## Waterbomb cell with 4 parameters (symmetric cell)

In [ ]:
wb = WBCell4Param()
wb.interact()